                Desenvolvimento de um Sistema de Gerenciamento de Riscos em Projetos de Software com Redes Bayesianas
Neste trabalho, foi desenvolvido um sistema de gerenciamento de riscos em projetos de software utilizando Redes Bayesianas. Essa abordagem permite modelar incertezas e relações causais entre variáveis relevantes ao sucesso ou fracasso de um projeto, oferecendo suporte à tomada de decisão baseada em probabilidade.

A rede construída possui oito variáveis principais: experiência da equipe, complexidade do projeto, ambiente de Desenvolvimento estável, escopo definido, qualidade do código,falha na entrega de funcionalidades, atraso no cronograma e aumento de custo.

As variáveis foram conectadas de acordo com suas relações de causa e efeito, representadas por um grafo acíclico direcionado (DAG). Por exemplo, a qualidade do código é influenciada pela experiência da equipe, pela estabilidade do ambiente e pela complexidade do projeto. A falha na entrega é influenciada pela qualidade, pela complexidade e pela clareza do escopo. Essas falhas, por sua vez, aumentam a chance de atraso, o qual pode gerar aumento de custos.

Cada nó da rede foi modelado com TabularCPDs (Tabelas de Probabilidade Condicional. A definição dessas tabelas possibilita que, a partir de evidências observadas (por exemplo: equipe inexperiente, escopo mal definido), o sistema seja capaz de inferir probabilidades para eventos de risco, como atrasos ou aumento de custos.

A inferência foi realizada por meio do algoritmo de eliminação de variáveis (Variable Elimination), implementado com a biblioteca pgmpy. Como exemplo de aplicação, o sistema foi capaz de estimar a probabilidade de aumento de custo dado um conjunto de condições críticas, como equipe inexperiente, ambiente instável e alta complexidade.


In [ ]:
!pip install pgmpy --upgrade

from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# estrutura da rede
model = DiscreteBayesianNetwork([
    ('ExperienciaEquipe', 'Qualidade'),
    ('AmbienteDesenvEstavel', 'Qualidade'),
    ('ComplexidadeProjeto', 'Qualidade'),
    ('ComplexidadeProjeto', 'Falha'),
    ('EscopoDefinido', 'Falha'),
    ('Qualidade', 'Falha'),
    ('Falha', 'Atraso'),
    ('Atraso', 'Custo')
])

In [ ]:
# tabelas de probabilidade independentes
cpd_experiencia = TabularCPD(variable='ExperienciaEquipe', variable_card=2, values=[[0.6], [0.4]])
cpd_complexidade = TabularCPD(variable='ComplexidadeProjeto', variable_card=2, values=[[0.5], [0.5]])
cpd_ambiente = TabularCPD(variable='AmbienteDesenvEstavel', variable_card=2, values=[[0.7], [0.3]])
cpd_escopo = TabularCPD(variable='EscopoDefinido', variable_card=2, values=[[0.8], [0.2]])

In [ ]:
# CPD da qualidade (3 pais: experiência, ambiente, complexidade)
cpd_qualidade = TabularCPD(
    variable='Qualidade', variable_card=2,
    values=[
        [0.95, 0.85, 0.75, 0.6, 0.7, 0.55, 0.45, 0.3],  # alta
        [0.05, 0.15, 0.25, 0.4, 0.3, 0.45, 0.55, 0.7]   # baixa
    ],
    evidence=['ExperienciaEquipe', 'AmbienteDesenvEstavel', 'ComplexidadeProjeto'],
    evidence_card=[2, 2, 2]
)

In [ ]:
# CPD da falha (3 pais: complexidade, escopo, qualidade)
cpd_falha = TabularCPD(
    variable='Falha', variable_card=2,
    values=[
        [0.95, 0.9, 0.85, 0.8, 0.75, 0.65, 0.55, 0.4],  # não
        [0.05, 0.1, 0.15, 0.2, 0.25, 0.35, 0.45, 0.6]   # sim
    ],
    evidence=['ComplexidadeProjeto', 'EscopoDefinido', 'Qualidade'],
    evidence_card=[2, 2, 2]
)

In [ ]:
# CPD do atraso (pai: falha)
cpd_atraso = TabularCPD(
    variable='Atraso', variable_card=2,
    values=[
        [0.8, 0.25],  # Não
        [0.2, 0.75]   # Sim
    ],
    evidence=['Falha'], evidence_card=[2]
)

In [ ]:
# CPD do Custo (pai: atraso)
cpd_custo = TabularCPD(
    variable='Custo', variable_card=2,
    values=[
        [0.8, 0.15],  # Não
        [0.2, 0.85]   # Sim
    ],
    evidence=['Atraso'], evidence_card=[2]
)

In [ ]:
# adicionando CPDs
model.add_cpds(
    cpd_experiencia, cpd_complexidade, cpd_ambiente, cpd_escopo,
    cpd_qualidade, cpd_falha, cpd_atraso, cpd_custo
)

In [ ]:
# verificando a rede
assert model.check_model()
print("Modelo valido")

In [ ]:
# Inferência
infer = VariableElimination(model)

In [ ]:
# exemplos
resultado1 = infer.query(variables=['Custo'], evidence={
    'ExperienciaEquipe': 1,
    'ComplexidadeProjeto': 1,
    'EscopoDefinido': 1,
    'AmbienteDesenvEstavel': 1
})
print("\n Probabilidade de Custo (com equipe inexperiente, projeto complexo, escopo indefinido e ambiente instável):")
print(resultado1)

resultado2 = infer.query(variables=['Atraso'], evidence={'Falha': 1})
print("\n Probabilidade de Atraso se houve falha na entrega:")
print(resultado2)

resultado3 = infer.query(variables=['Qualidade'], evidence={
    'ExperienciaEquipe': 0,
    'AmbienteDesenvEstavel': 0,
    'ComplexidadeProjeto': 0
})
print("\n Probabilidade de Qualidade com equipe experiente, ambiente estável e projeto simples:")
print(resultado3)

QUESTÃO 5

In [18]:
base_de_casos = [
    {"sintomas": {"febre", "tosse", "dor de garganta"}, "diagnostico": "Gripe"},
    {"sintomas": {"dor de cabeça", "sensibilidade à luz", "náusea"}, "diagnostico": "Enxaqueca"},
    {"sintomas": {"dor abadominal", "náusea", "diarreia"}, "diagnostico": "Gastroenterite"},
    {"sintomas": {"febre", "dor muscular", "cansaço"}, "diagnostico": "Dengue"},
    {"sintomas": {"tosse", "falta de ar", "dor no peito"}, "diagnostico": "Bronquite"}
]

In [19]:
def comparar_sintomas(sintomas_paciente, base):
    melhores_casos = []
    maior_similaridade = 0

    for caso in base:
        intersecao = sintomas_paciente.intersection(caso["sintomas"])
        similaridade = len(intersecao)

        if similaridade > maior_similaridade:
            melhores_casos = [caso]
            maior_similaridade = similaridade
        elif similaridade == maior_similaridade and similaridade > 0:
            melhores_casos.append(caso)

    return melhores_casos, maior_similaridade


In [28]:
# sintomas do paciente
#sintomas_paciente = {"febre", "tosse", "dor de garganta", "dor de cabeça"}
sintomas_paciente = {"febre"}

# faz a busca

casos_similares, similaridade = comparar_sintomas(sintomas_paciente, base_de_casos)

In [29]:
#mostra os resultados

if casos_similares:
    print(f"Casos mais semelhantes encontrados com {similaridade} sintomas em comum:\n")
    for i, caso in enumerate(casos_similares, 1):
        #print(f"{i}. Diagnóstico sugerido: {caso['diagnostico']} com os sintomas: {caso['sintomas']}")
        print(f"{i}. Diagnóstico sugerido: {caso['diagnostico']}")
else:
    print("\nNenhum caso similar encontrado.")


Casos mais semelhantes encontrados com 1 sintomas em comum:

1. Diagnóstico sugerido: Gripe
2. Diagnóstico sugerido: Dengue
